In [1]:
"""
Tuner：帮助选择最佳的超参数集
1.模型超参数：影响模型（隐藏层数量、宽度)
2.算法超参数：影响算法学习速度与质量（学习率等）
"""
import tensorflow as tf
from tensorflow import keras

In [5]:
!pip install --user keras-tuner
import keras_tuner as kt

  Using cached keras_tuner-1.3.3-py3-none-any.whl (172 kB)


  Using cached kt_legacy-1.0.4-py3-none-any.whl (9.6 kB)
  Using cached gast-0.3.3-py2.py3-none-any.whl (9.7 kB)
  Using cached six-1.15.0-py2.py3-none-any.whl (10 kB)
  Using cached numpy-1.19.5-cp37-cp37m-manylinux2010_x86_64.whl (14.8 MB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
pytest-astropy 0.8.0 requires pytest-filter-subpackage>=0.1, which is not installed.
virtualenv 20.0.25 requires importlib-metadata<2,>=0.12; python_version < "3.8", but you have importlib-metadata 4.6.4 which is incompatible.
tensorflow-gpu 1.13.1 requires tensorboard<1.14.0,>=1.13.0, but you have tensorboard 2.11.2 which is incompatible.
tensorflow-gpu 1.13.1 requires tensorflow-estimator<1.14.0rc0,>=1.13.0, but you have tensorflow-estimator 2.4.0 which is incompatible.
pyldavis 3.3.1 requires numpy>=1.20.0, but you have numpy 1.19.5 which is incompatible.
paddlehub 1.6.1 requires yapf==0.26.0, but you have yapf 0.31.0 which is incompatible.
astroid 2.3.3 requires lazy-object-proxy==1.4.*, but you have lazy-object-proxy 1.6.0 which is incompa

In [4]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

4423680/4422102 [==============================] - 1s 0us/step


In [18]:
"""
除了模型架构外，还需要构建超参数的搜索空间，为超参数调整设计模型—— 超模型
两种方式定义超模型：
1.使用模型构建工具函数
2.将keras Tuner API的HyperModel类子类化
"""
# 将使用模型构建工具函数来定义图像分类模型。
# 模型构建工具函数将返回已编译的模型，并使用您以内嵌方式定义的超参数对模型进行超调
def model_builder(hp):
    model = keras.Sequential()
    # Flatten:压缩为1维数据，用于全连接层输入
    model.add(keras.layers.Flatten(input_shape=(28,28)))
    
    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    # unit:输出维度
    model.add(keras.layers.Dense(units=hp_units,activation='relu'))
    model.add(keras.layers.Dense(10))
    
    # Tune the learning rate for the optimizer
    # choose an optimal value from 0.01, 0.001, 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2,1e-3,1e-4])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),       
                  metrics=['accuracy'])
    return model

In [22]:
"""
实例化调节器，并实现超调
Kears Tuner提供了四种超调器：RandomSearch, Hyperband, BayesianOptimization, Sklearn
实例化调节器：指定超模型、要优化的objective、最大训练周期数
"""
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

INFO:tensorflow:Reloading Tuner from my_dir/intro_to_kt/tuner0.json


In [23]:
"""
Hyperband 调节算法使用自适应资源分配和早停法来快速收敛到高性能模型。
该过程采用了体育竞技争冠模式的排除法。
算法会将大量模型训练多个周期，并仅将性能最高的一半模型送入下一轮训练。
Hyperband 通过计算 1 + log_factor(max_epochs) 
并将其向上舍入到最接近的整数来确定要训练的模型的数量。
"""
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
# 运行超参数搜索
tuner.search(img_train, label_train, epochs=30, validation_split=0.2, callbacks=[stop_early])

# 获取最优超参数
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


Search: Running Trial #7

Value             |Best Value So Far |Hyperparameter
160               |160               |units
0.001             |0.001             |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Train on 48000 samples, validate on 12000 samples
Epoch 1/2
47872/48000 [============================>.] - ETA: 0s - loss: 0.5117 - accuracy: 0.8198

Traceback (most recent call last):
  File "/home/zhuhou2/.local/lib/python3.7/site-packages/keras_tuner/engine/base_tuner.py", line 266, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/home/zhuhou2/.local/lib/python3.7/site-packages/keras_tuner/engine/base_tuner.py", line 231, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/home/zhuhou2/.local/lib/python3.7/site-packages/keras_tuner/tuners/hyperband.py", line 425, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
  File "/home/zhuhou2/.local/lib/python3.7/site-packages/keras_tuner/engine/tuner.py", line 287, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/home/zhuhou2/.local/lib/python3.7/site-packages/keras_tuner/engine/tuner.py", line 214, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
  File "/home/zhuhou2/.local/lib/python3.7

RuntimeError: Number of consecutive failures excceeded the limit of 3.
Traceback (most recent call last):
  File "/home/zhuhou2/.local/lib/python3.7/site-packages/keras_tuner/engine/base_tuner.py", line 266, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/home/zhuhou2/.local/lib/python3.7/site-packages/keras_tuner/engine/base_tuner.py", line 231, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/home/zhuhou2/.local/lib/python3.7/site-packages/keras_tuner/tuners/hyperband.py", line 425, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
  File "/home/zhuhou2/.local/lib/python3.7/site-packages/keras_tuner/engine/tuner.py", line 287, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/home/zhuhou2/.local/lib/python3.7/site-packages/keras_tuner/engine/tuner.py", line 214, in _build_and_fit_model
    results = self.hypermodel.fit

In [21]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

NameError: name 'best_hps' is not defined

In [ ]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)